In [ ]:
# ============================================================================
# CRITICAL FIX: Must be executed FIRST before any other imports
# ============================================================================
import os
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

import sys
try:
    import google.protobuf
    if hasattr(google.protobuf, '__version__'):
        print(f"Protobuf version: {google.protobuf.__version__}")
except:
    pass

print("Environment variables set successfully")
print("="*60)

In [ ]:
# Core imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

seed = 42
np.random.seed(seed)

print("="*60)
print("IMPORTING TENSORFLOW AND KERAS...")
print("="*60)

import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Embedding, LSTM, Bidirectional, Dense, Dropout, 
    GlobalMaxPooling1D, Conv1D, SpatialDropout1D
)
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2

tf.random.set_seed(seed)
keras.utils.set_random_seed(seed)

from sklearn.metrics import matthews_corrcoef, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

pd.set_option('display.max_rows', 36)
pd.set_option("display.max_colwidth", 150)

print("="*60)
print("ENVIRONMENT SETUP - SUCCESS")
print("="*60)
print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")
print(f"GPU Available: {len(tf.config.list_physical_devices('GPU')) > 0}")
if tf.config.list_physical_devices('GPU'):
    gpu_devices = tf.config.list_physical_devices('GPU')
    print(f"GPU Devices: {len(gpu_devices)} device(s)")
    for gpu in gpu_devices:
        print(f"  - {gpu.name}")
print(f"Random seed: {seed}")
print("="*60)

## Iteración 3 - Configuración del Modelo - TERAPIA DE CHOQUE

**Problema Persistente:**
- V1: Overfitting Delta = 0.184 (MCC 0.8665)
- V2: Overfitting Delta = 0.166 (MCC 0.8885) - Mejora insuficiente 9.6%

**Terapia de Choque - Cambios Radicales:**
1. Reducir capacidad modelo 50% total vs V1
2. Dropout extremo (0.7 en dense, 0.4 spatial)
3. L2 regularization x5 más fuerte
4. Learning rate más bajo (control de gradientes)
5. Gradient clipping (evitar explosión)
6. Early stopping ultra agresivo (patience=2)

**Meta:** Overfitting Delta < 0.08 manteniendo MCC > 0.87

In [ ]:
# Hyperparameters Configuration - ITERACIÓN 3 - TERAPIA DE CHOQUE
MAX_WORDS = 10000
MAX_LEN = 200
EMBEDDING_DIM = 100

# Model Architecture - CAMBIOS AGRESIVOS
LSTM_UNITS = 64           # ITER3: 96 → 64 (-33% vs V2, -50% vs V1)
DENSE_UNITS = 32          # ITER3: 48 → 32 (-33% vs V2, -50% vs V1)
DROPOUT_RATE = 0.7        # ITER3: 0.6 → 0.7 (EXTREMO)
SPATIAL_DROPOUT = 0.4     # ITER3: 0.3 → 0.4 (EXTREMO)
L2_REG = 5e-4             # ITER3: 1e-4 → 5e-4 (x5 más fuerte)

# Training Configuration - AJUSTES AGRESIVOS
BATCH_SIZE = 32
EPOCHS = 50
VALIDATION_SPLIT = 0.2
LEARNING_RATE = 5e-4      # ITER3: 1e-3 → 5e-4 (reduce velocidad)
CLIPNORM = 1.0            # ITER3: NUEVO - Gradient clipping

print("="*60)
print("MODEL CONFIGURATION - ITERACIÓN 3")
print("TERAPIA DE CHOQUE - OVERFITTING KILLER")
print("="*60)
print("OBJETIVO: Overfitting Delta < 0.08 y MCC > 0.87")
print("="*60)
print("CAMBIOS AGRESIVOS respecto a V2:")
print("  - LSTM Units: 96 → 64 (-33%)")
print("  - Dense Units: 48 → 32 (-33%)")
print("  - Spatial Dropout: 0.3 → 0.4 (EXTREMO)")
print("  - Dropout: 0.6 → 0.7 (EXTREMO)")
print("  - L2 Reg: 1e-4 → 5e-4 (x5 más fuerte)")
print("  - Learning Rate: 1e-3 → 5e-4 (más lento)")
print("  - Gradient Clipping: None → 1.0 (NUEVO)")
print("  - Early Stop Patience: 3 → 2 (ultra agresivo)")
print("="*60)
print(f"Vocabulary Size: {MAX_WORDS:,}")
print(f"Sequence Length: {MAX_LEN}")
print(f"LSTM Units: {LSTM_UNITS}")
print(f"Dense Units: {DENSE_UNITS}")
print(f"Dropout Rate: {DROPOUT_RATE}")
print(f"Spatial Dropout: {SPATIAL_DROPOUT}")
print(f"L2 Regularization: {L2_REG}")
print(f"Learning Rate: {LEARNING_RATE}")
print(f"Gradient Clip Norm: {CLIPNORM}")
print("="*60)

# Comparación de parámetros
estimated_params_v3 = (
    MAX_WORDS * EMBEDDING_DIM +
    4 * LSTM_UNITS * (EMBEDDING_DIM + LSTM_UNITS + 1) * 2 +
    (LSTM_UNITS * 2) * DENSE_UNITS + DENSE_UNITS +
    DENSE_UNITS + 1
)
print(f"\nComparación de parámetros:")
print(f"   V1: 1,251,009 parámetros")
print(f"   V2: 1,160,609 parámetros (-7.2% vs V1)")
print(f"   V3: ~{estimated_params_v3:,} parámetros (estimado)")
print(f"   Reducción total V3 vs V1: ~{((1251009 - estimated_params_v3) / 1251009 * 100):.1f}%")
print("="*60)

## Carga y Exploración de Datos

In [ ]:
# Load training data
train = pd.read_csv("/kaggle/input/u-tad-spam-not-spam-2025-edition/train.csv", index_col="row_id")

print("="*60)
print("TRAINING DATA OVERVIEW")
print("="*60)
print(f"Total samples: {len(train):,}")
print(f"\nColumns: {list(train.columns)}")
print(f"\nClass distribution:\n{train['spam_label'].value_counts()}")
print(f"\nClass balance:\n{train['spam_label'].value_counts(normalize=True)}")
print("="*60)

train.head(10)

In [ ]:
# Análisis de longitud de textos
train['text_length'] = train['text'].apply(lambda x: len(str(x).split()))

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.hist(train['text_length'], bins=50, edgecolor='black', alpha=0.7)
plt.title('Distribución de Longitud de Textos')
plt.xlabel('Número de palabras')
plt.ylabel('Frecuencia')
plt.axvline(train['text_length'].mean(), color='red', linestyle='--', label=f'Media: {train["text_length"].mean():.1f}')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
sns.boxplot(data=train, x='spam_label', y='text_length')
plt.title('Longitud por Clase')
plt.xlabel('Clase (0=Not SPAM, 1=SPAM)')
plt.ylabel('Número de palabras')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Longitud promedio: {train['text_length'].mean():.2f} palabras")
print(f"Longitud mediana: {train['text_length'].median():.2f} palabras")

## Preprocesamiento de Texto

In [ ]:
# Preparar textos y etiquetas
X_train_text = train['text'].values
y_train = train['spam_label'].values

# Tokenización
print("Tokenizando textos...")
tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train_text)

X_train_seq = tokenizer.texts_to_sequences(X_train_text)
X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post', truncating='post')

# Split train/validation
X_train_final, X_val, y_train_final, y_val = train_test_split(
    X_train_pad, y_train, 
    test_size=VALIDATION_SPLIT, 
    random_state=seed,
    stratify=y_train
)

print("="*60)
print("PREPROCESSING SUMMARY")
print("="*60)
print(f"Vocabulary size: {len(tokenizer.word_index):,}")
print(f"Training samples: {len(X_train_final):,}")
print(f"Validation samples: {len(X_val):,}")
print(f"Sequence shape: {X_train_pad.shape}")
print("="*60)

## Construcción del Modelo - ITERACIÓN 3 - TERAPIA DE CHOQUE

**Arquitectura Ultra-Regularizada:**
- Embedding → SpatialDropout(0.4) → Bi-LSTM(64) + L2(5e-4) → GlobalMaxPool → Dense(32) + L2(5e-4) → Dropout(0.7) → Output

**Estrategia:**
- Capacidad reducida 50% vs V1
- Regularización extrema en todas las capas
- Gradient clipping para estabilidad
- Learning rate bajo para convergencia suave

In [ ]:
# Construcción del modelo V3 - TERAPIA DE CHOQUE
def build_model_v3():
    model = Sequential([
        # Embedding layer
        Embedding(
            input_dim=MAX_WORDS,
            output_dim=EMBEDDING_DIM,
            input_length=MAX_LEN,
            name='embedding'
        ),
        
        # ITER3: Spatial Dropout EXTREMO 0.4
        SpatialDropout1D(SPATIAL_DROPOUT),
        
        # ITER3: LSTM reducido a 64 units + L2 x5 más fuerte
        Bidirectional(
            LSTM(
                LSTM_UNITS, 
                return_sequences=True,
                kernel_regularizer=l2(L2_REG),
                recurrent_regularizer=l2(L2_REG),
                bias_regularizer=l2(L2_REG)     # NUEVO: regularizar bias también
            ), 
            name='bidirectional_lstm'
        ),
        
        GlobalMaxPooling1D(),
        
        # ITER3: Dense reducido a 32 units + L2 x5 más fuerte
        Dense(
            DENSE_UNITS, 
            activation='relu',
            kernel_regularizer=l2(L2_REG),
            bias_regularizer=l2(L2_REG),        # NUEVO: regularizar bias también
            name='dense_1'
        ),
        
        # ITER3: Dropout EXTREMO 0.7
        Dropout(DROPOUT_RATE),
        
        # Output layer
        Dense(1, activation='sigmoid', name='output')
    ], name='spam_classifier_v3_shock_therapy')
    
    return model

# Crear modelo
model = build_model_v3()

# ITER3: AdamW con LR reducido + Gradient Clipping
optimizer = keras.optimizers.AdamW(
    learning_rate=LEARNING_RATE,
    weight_decay=1e-4,
    clipnorm=CLIPNORM  # NUEVO: Gradient clipping
)

model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        keras.metrics.Precision(name='precision'),
        keras.metrics.Recall(name='recall'),
        keras.metrics.AUC(name='auc')
    ]
)

model.build(input_shape=(None, MAX_LEN))
model.summary()

print("\n" + "="*60)
print("MODEL V3 COMPILED - SHOCK THERAPY MODE")
print("="*60)
print(f"Total parameters: {model.count_params():,}")
print(f"\nComparación:")
print(f"   V1: 1,251,009 params | MCC 0.8665 | Overfitting Δ 0.184")
print(f"   V2: 1,160,609 params | MCC 0.8885 | Overfitting Δ 0.166")
print(f"   V3: {model.count_params():,} params")
print(f"\n   Reducción V3 vs V1: {((1251009 - model.count_params()) / 1251009 * 100):.1f}%")
print(f"   Reducción V3 vs V2: {((1160609 - model.count_params()) / 1160609 * 100):.1f}%")
print(f"\nOptimizer: AdamW (lr={LEARNING_RATE}, clipnorm={CLIPNORM})")
print(f"L2 Regularization: {L2_REG} (x5 más fuerte que V2)")
print(f"\nOBJETIVO: Overfitting Delta < 0.08 y MCC > 0.87")
print("="*60)

## Entrenamiento del Modelo - ITERACIÓN 3

**Callbacks Ultra-Agresivos:**
- EarlyStopping: patience=2 (detiene al mínimo signo de estancamiento)
- ReduceLROnPlateau: patience=1 (reduce LR inmediatamente)
- ModelCheckpoint: guarda mejor modelo

In [ ]:
# ITER3: Callbacks ULTRA-AGRESIVOS
callbacks = [
    # ITER3: Patience 2 (ultra agresivo)
    EarlyStopping(
        monitor='val_loss',
        patience=2,  # CAMBIO: 3 → 2
        restore_best_weights=True,
        verbose=1
    ),
    ModelCheckpoint(
        'best_spam_model_v3.keras',
        monitor='val_loss',
        save_best_only=True,
        verbose=1
    ),
    # ITER3: Patience 1 (reduce LR inmediatamente)
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=1,  # CAMBIO: 2 → 1
        min_lr=1e-6,
        verbose=1
    )
]

print("="*60)
print("STARTING TRAINING - ITERACIÓN 3 - SHOCK THERAPY")
print("="*60)
print("Monitoreando overfitting con configuración ultra-agresiva...")
print("META: Train Loss > 0.05 | Val Loss < 0.14 | Delta < 0.08")
print("="*60)

history = model.fit(
    X_train_final, y_train_final,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(X_val, y_val),
    callbacks=callbacks,
    verbose=1
)

print("\n" + "="*60)
print("TRAINING COMPLETED - ITERACIÓN 3")
print("="*60)

## Evaluación del Modelo

In [ ]:
# Predicciones en validación
y_pred_proba = model.predict(X_val)
y_pred = (y_pred_proba > 0.5).astype(int).flatten()

mcc_score = matthews_corrcoef(y_val, y_pred)

print("="*60)
print("VALIDATION METRICS - ITERACIÓN 3")
print("="*60)
print(f"Matthews Correlation Coefficient: {mcc_score:.4f}")
print("\nClassification Report:")
print(classification_report(y_val, y_pred, target_names=['Not SPAM', 'SPAM']))
print("="*60)

# Matriz de confusión
plt.figure(figsize=(8, 6))
cm = confusion_matrix(y_val, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Reds', 
            xticklabels=['Not SPAM', 'SPAM'],
            yticklabels=['Not SPAM', 'SPAM'])
plt.title(f'Confusion Matrix V3 - Shock Therapy (MCC: {mcc_score:.4f})')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

## Análisis Crítico - V1 vs V2 vs V3

In [ ]:
# Análisis comparativo completo
final_train_loss = history.history['loss'][-1]
final_val_loss = history.history['val_loss'][-1]
final_train_acc = history.history['accuracy'][-1]
final_val_acc = history.history['val_accuracy'][-1]
overfitting_delta = abs(final_val_loss - final_train_loss)

print("\n" + "="*80)
print("ANÁLISIS CRÍTICO - EVOLUCIÓN V1 → V2 → V3")
print("="*80)

# Datos de iteraciones anteriores
v1_data = {'train_loss': 0.0055, 'val_loss': 0.1895, 'mcc': 0.8665, 'params': 1251009}
v2_data = {'train_loss': 0.0411, 'val_loss': 0.2075, 'mcc': 0.8885, 'params': 1160609}
v3_data = {
    'train_loss': final_train_loss, 
    'val_loss': final_val_loss, 
    'mcc': mcc_score,
    'params': model.count_params()
}

v1_delta = abs(v1_data['val_loss'] - v1_data['train_loss'])
v2_delta = abs(v2_data['val_loss'] - v2_data['train_loss'])
v3_delta = overfitting_delta

print("\nITERACIÓN 1 (Baseline):")
print(f"   Train Loss: {v1_data['train_loss']:.4f} | Val Loss: {v1_data['val_loss']:.4f}")
print(f"   Overfitting Delta: {v1_delta:.4f}")
print(f"   MCC: {v1_data['mcc']:.4f} | Params: {v1_data['params']:,}")

print("\nITERACIÓN 2 (Regularización Moderada):")
print(f"   Train Loss: {v2_data['train_loss']:.4f} | Val Loss: {v2_data['val_loss']:.4f}")
print(f"   Overfitting Delta: {v2_delta:.4f}")
print(f"   MCC: {v2_data['mcc']:.4f} | Params: {v2_data['params']:,}")
print(f"   Mejora overfitting: {((v1_delta - v2_delta) / v1_delta * 100):.1f}%")
print(f"   Mejora MCC: +{(v2_data['mcc'] - v1_data['mcc']):.4f}")

print("\nITERACIÓN 3 (Terapia de Choque):")
print(f"   Train Loss: {v3_data['train_loss']:.4f} | Val Loss: {v3_data['val_loss']:.4f}")
print(f"   Overfitting Delta: {v3_delta:.4f}")
print(f"   MCC: {v3_data['mcc']:.4f} | Params: {v3_data['params']:,}")
print(f"   Mejora overfitting vs V2: {((v2_delta - v3_delta) / v2_delta * 100):.1f}%")
print(f"   Mejora overfitting vs V1: {((v1_delta - v3_delta) / v1_delta * 100):.1f}%")
print(f"   Cambio MCC vs V2: {(v3_data['mcc'] - v2_data['mcc']):.4f}")
print(f"   Cambio MCC vs V1: {(v3_data['mcc'] - v1_data['mcc']):.4f}")

print("\n" + "="*80)
print("EVALUACIÓN FINAL:")
print("="*80)

success_v3 = [
    ("Overfitting Delta < 0.08", v3_delta < 0.08),
    ("MCC >= 0.87", v3_data['mcc'] >= 0.87),
    ("Val Loss < 0.14", v3_data['val_loss'] < 0.14),
    ("Train Loss > 0.05", v3_data['train_loss'] > 0.05),
    ("Mejora vs V2", v3_delta < v2_delta)
]

for criterion, success in success_v3:
    status = "OK" if success else "FAIL"
    print(f"[{status}] {criterion}")

total_success = sum([s for _, s in success_v3])
print("\n" + "="*80)
print(f"CRITERIOS CUMPLIDOS: {total_success}/5")

if total_success >= 4:
    print("\nRESULTADO: TERAPIA DE CHOQUE EXITOSA")
    print("PRÓXIMO PASO: Iteración 4 con GloVe embeddings pre-entrenados")
elif total_success >= 3:
    print("\nRESULTADO: MEJORA SIGNIFICATIVA")
    print("PRÓXIMO PASO: Ajustar threshold o probar arquitecturas alternativas")
else:
    print("\nRESULTADO: INSUFICIENTE - Considerar cambio de estrategia")
    print("PRÓXIMO PASO: CNN+LSTM híbrido o arquitectura completamente diferente")

print("="*80)

## Curvas de Aprendizaje

In [ ]:
# Visualización de curvas
def plot_learning_curves(history, title_prefix=""):
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Loss
    axes[0, 0].plot(history.history['loss'], label='Train Loss', linewidth=2, color='red')
    axes[0, 0].plot(history.history['val_loss'], label='Val Loss', linewidth=2, color='darkred')
    axes[0, 0].set_title(f'{title_prefix} Loss', fontsize=12, fontweight='bold')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # Accuracy
    axes[0, 1].plot(history.history['accuracy'], label='Train Acc', linewidth=2, color='red')
    axes[0, 1].plot(history.history['val_accuracy'], label='Val Acc', linewidth=2, color='darkred')
    axes[0, 1].set_title(f'{title_prefix} Accuracy', fontsize=12, fontweight='bold')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Accuracy')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # Precision
    axes[1, 0].plot(history.history['precision'], label='Train Prec', linewidth=2, color='red')
    axes[1, 0].plot(history.history['val_precision'], label='Val Prec', linewidth=2, color='darkred')
    axes[1, 0].set_title(f'{title_prefix} Precision', fontsize=12, fontweight='bold')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Precision')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # Recall
    axes[1, 1].plot(history.history['recall'], label='Train Recall', linewidth=2, color='red')
    axes[1, 1].plot(history.history['val_recall'], label='Val Recall', linewidth=2, color='darkred')
    axes[1, 1].set_title(f'{title_prefix} Recall', fontsize=12, fontweight='bold')
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('Recall')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

plot_learning_curves(history, title_prefix="V3 Shock Therapy")

## Predicciones en Test Data

In [ ]:
# Load test data
test = pd.read_csv("/kaggle/input/u-tad-spam-not-spam-2025-edition/test.csv", index_col="row_id")

print("="*60)
print("TEST DATA")
print("="*60)
print(f"Total test samples: {len(test):,}")
print("="*60)

test.head()

In [ ]:
# Preprocesar y predecir
X_test_text = test['text'].values
X_test_seq = tokenizer.texts_to_sequences(X_test_text)
X_test_pad = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post', truncating='post')

print("Generando predicciones en test data...")
y_pred_proba_test = model.predict(X_test_pad, batch_size=BATCH_SIZE)
y_pred_test = (y_pred_proba_test > 0.5).astype(int).flatten()

print(f"Predicciones generadas: {len(y_pred_test):,}")
print(f"Distribución:")
print(f"  Not SPAM: {np.sum(y_pred_test == 0):,} ({np.mean(y_pred_test == 0)*100:.2f}%)")
print(f"  SPAM: {np.sum(y_pred_test == 1):,} ({np.mean(y_pred_test == 1)*100:.2f}%)")

## Generación de Submission

In [ ]:
# Crear submission file
submission = pd.read_csv("/kaggle/input/u-tad-spam-not-spam-2025-edition/sample_submission.csv")
submission["spam_label"] = y_pred_test
submission.to_csv('submission.csv', index=False)

print("="*60)
print("SUBMISSION FILE CREATED - ITERACIÓN 3")
print("="*60)
print(f"Total predictions: {len(submission):,}")
print(f"File: submission.csv")
print("="*60)

submission.head(10)

## Resumen Final Comparativo

In [ ]:
# Tabla comparativa final
comparison_df = pd.DataFrame({
    'Métrica': [
        'Validation MCC',
        'Train Loss',
        'Val Loss',
        'Overfitting Delta',
        'Train Accuracy',
        'Val Accuracy',
        'Total Parameters',
        'Training Epochs'
    ],
    'V1 Baseline': [
        '0.8665',
        '0.0055',
        '0.1895',
        '0.1840',
        '99.91%',
        '95.77%',
        '1,251,009',
        '8'
    ],
    'V2 Moderate': [
        '0.8885',
        '0.0411',
        '0.2075',
        '0.1663',
        '99.56%',
        '95.07%',
        '1,160,609',
        '6'
    ],
    'V3 Shock': [
        f'{mcc_score:.4f}',
        f'{final_train_loss:.4f}',
        f'{final_val_loss:.4f}',
        f'{overfitting_delta:.4f}',
        f'{final_train_acc:.2%}',
        f'{final_val_acc:.2%}',
        f'{model.count_params():,}',
        f'{len(history.history["loss"])}'
    ]
})

print("="*90)
print("TABLA COMPARATIVA FINAL - V1 vs V2 vs V3")
print("="*90)
print(comparison_df.to_string(index=False))
print("="*90)

print("\nEVOLUCIÓN OVERFITTING:")
print(f"  V1: 0.1840 (baseline)")
print(f"  V2: 0.1663 (-9.6%)")
print(f"  V3: {overfitting_delta:.4f} ({((0.1840 - overfitting_delta) / 0.1840 * 100):.1f}% vs V1)")

print("\nEVOLUCIÓN MCC:")
print(f"  V1: 0.8665 (baseline)")
print(f"  V2: 0.8885 (+0.0220)")
print(f"  V3: {mcc_score:.4f} ({'+' if mcc_score > 0.8665 else ''}{(mcc_score - 0.8665):.4f} vs V1)")

print("="*90)